# ResNet for MNIST in PyTorch

In [1]:
!git clone https://github.com/amirgholami/adahessian.git

Cloning into 'adahessian'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 494 (delta 48), reused 62 (delta 24), pack-reused 387
Receiving objects: 100% (494/494), 1.98 MiB | 21.38 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [9]:
# !npx degit amirgholami/adahessian -f

npx: installed 1 in 1.655s
> destination directory is not empty. Using --force, continuing
> cloned amirgholami/adahessian#master


In [27]:
import adahessian.image_classification.optim_adahessian

In [16]:
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time

import torch

from torch import nn, optim
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader
from copy import copy, deepcopy
import numpy as np

# from image_classification.optim_adahessian import Adahessian
# import adahessian


In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
def save_last_model(input_model):
  model_save_name = 'resnet_adahessian_last.pkl'
  path = F"/content/gdrive/My Drive/{model_save_name}" 
  torch.save(input_model, path)

In [20]:
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(MnistResNet, self).forward(x), dim=-1)


In [21]:
# def get_data_loaders(train_batch_size, val_batch_size):
#     mnist = MNIST(download=True, train=True, root=".").train_data.float()
    
#     # add gaussian noise maybe
#     data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])

#     train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
#                               batch_size=train_batch_size, shuffle=True)

#     val_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
#                             batch_size=val_batch_size, shuffle=False)
#     return train_loader, val_loader
def getData(name='cifar10', train_bs=128, test_bs=1000):    
    if name == 'svhn':
        train_loader = torch.utils.data.DataLoader(
    datasets.SVHN('../data', split='extra', download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
    datasets.SVHN('../data', split='test', download=True,transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=test_bs, shuffle=False)
    if name == 'mnist':
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=test_bs, shuffle=False)
    if name == 'emnist':
        train_loader = torch.utils.data.DataLoader(
            datasets.EMNIST('../data', train=True, download=True, split='balanced',
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1751,), (0.3267,))
                           ])),
            batch_size=train_bs, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            datasets.EMNIST('../data', train=False, split='balanced', transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1751,), (0.3267,))
                           ])),
            batch_size=test_bs, shuffle=False)
    if name == 'cifar10':
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        trainset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_bs, shuffle=True)
        testset = datasets.CIFAR10(root='../data', train=False, download=False, transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_bs, shuffle=False)
    if name == 'cifar100':
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        trainset = datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_bs, shuffle=True)
        testset = datasets.CIFAR100(root='../data', train=False, download=False, transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_bs, shuffle=False)
    if name == 'tinyimagenet':      
        normalize = transforms.Normalize(mean=[0.44785526394844055, 0.41693055629730225, 0.36942949891090393],
                                     std=[0.2928885519504547, 0.28230994939804077, 0.2889912724494934])
        train_dataset = datasets.ImageFolder(
        '../data/tiny-imagenet-200/train',
        transforms.Compose([
            transforms.RandomCrop(64, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_bs, shuffle=True, num_workers=4, pin_memory=False)
        test_dataset = datasets.ImageFolder(
        '../data/tiny-imagenet-200/val',
        transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]))
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_bs, shuffle=False)
    return train_loader, test_loader

In [22]:
def calculate_metric(metric_fn, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [29]:
# optimizer = optim.Adam(model.parameters())
adahessian.image_classification.optim_adahessian.Adahessian

adahessian.image_classification.optim_adahessian.Adahessian

In [31]:
start_ts = time.time()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 20

model = MnistResNet().to(device)
train_loader, val_loader = getData(name='mnist', train_bs=128, test_bs=1000)

losses = []
loss_function = nn.CrossEntropyLoss()
optimizer = adahessian.image_classification.optim_adahessian.Adahessian(model.parameters())

batches = len(train_loader)
val_batches = len(val_loader)
# keep best model
accuracies=[]
best_accuracy = 0
best_model = deepcopy(model)

# training loop + eval loop
for epoch in range(epochs):
    total_loss = 0
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)
    model.train()
    # # lr decay
    # optimizer = exp_lr_scheduler(epoch, optimizer, decay_eff=0.1, decayEpoch=[15])

    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)

        loss.backward(create_graph=True)  # You need this line for Hessian backprop
        optimizer.step()
        current_loss = loss.item()
        total_loss += current_loss
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    torch.cuda.empty_cache()
    
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            outputs = model(X)
            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1]
            
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
    
    current_model_accuracy = sum(accuracy)/val_batches
    accuracies.append(current_model_accuracy)
    if current_model_accuracy > best_accuracy:
        best_model = deepcopy(model)
        best_accuracy=current_model_accuracy
        
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)
    print('current_model_accuracy: ',current_model_accuracy)
    print('best_accuracy: ',best_accuracy)

save_last_model(model)
model_save_name = 'resnet_adahessian_best.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(best_model, path)

print(losses)
print(f"Training time: {time.time()-start_ts}s")


Epoch 1/20, training loss: 1.531552266972914, validation loss: 1.4988209009170532
	     precision: 0.9642
	        recall: 0.9620
	            F1: 0.9618
	      accuracy: 0.9621
current_model_accuracy:  0.9621000000000001
best_accuracy:  0.9621000000000001



Epoch 2/20, training loss: 1.4942002687881242, validation loss: 1.4905128479003906
	     precision: 0.9724
	        recall: 0.9710
	            F1: 0.9710
	      accuracy: 0.9712
current_model_accuracy:  0.9712
best_accuracy:  0.9712



Epoch 3/20, training loss: 1.492943156248471, validation loss: 1.4872047901153564
	     precision: 0.9742
	        recall: 0.9738
	            F1: 0.9735
	      accuracy: 0.9738
current_model_accuracy:  0.9738
best_accuracy:  0.9738



Epoch 4/20, training loss: 1.4882341913068726, validation loss: 1.4847815036773682
	     precision: 0.9779
	        recall: 0.9765
	            F1: 0.9766
	      accuracy: 0.9768
current_model_accuracy:  0.9767999999999999
best_accuracy:  0.9767999999999999



Epoch 5/20, training loss: 1.4861529605475061, validation loss: 1.4866466522216797
	     precision: 0.9756
	        recall: 0.9746
	            F1: 0.9747
	      accuracy: 0.9748
current_model_accuracy:  0.9747999999999999
best_accuracy:  0.9767999999999999


...


Epoch 16/20, training loss: 1.4785077457488982, validation loss: 1.4872995615005493
	     precision: 0.9745
	        recall: 0.9735
	            F1: 0.9735
	      accuracy: 0.9737
current_model_accuracy:  0.9737
best_accuracy:  0.9863



Epoch 17/20, training loss: 1.4761183971026812, validation loss: 1.4753059148788452
	     precision: 0.9860
	        recall: 0.9855
	            F1: 0.9855
	      accuracy: 0.9857
current_model_accuracy:  0.9857000000000001
best_accuracy:  0.9863



Epoch 18/20, training loss: 1.4772437043281506, validation loss: 1.4766126871109009
	     precision: 0.9844
	        recall: 0.9843
	            F1: 0.9842
	      accuracy: 0.9844
current_model_accuracy:  0.9844000000000002
best_accuracy:  0.9863



Epoch 19/20, training loss: 1.4762220685161762, validation loss: 1.4800760746002197
	     precision: 0.9816
	        recall: 0.9802
	            F1: 0.9805
	      accuracy: 0.9809
current_model_accuracy:  0.9808999999999999
best_accuracy:  0.9863



Epoch 20/20, training loss: 1.4771273441152024, validation loss: 1.4751309156417847
	     precision: 0.9862
	        recall: 0.9860
	            F1: 0.9859
	      accuracy: 0.9861
current_model_accuracy:  0.9861000000000001
best_accuracy:  0.9863
[1.531552266972914, 1.4942002687881242, 1.492943156248471, 1.4882341913068726, 1.4861529605475061, 1.4848912992457082, 1.4812691760724033, 1.4803544862438112, 1.4824339563150142, 1.4808943075920218, 1.480138957373369, 1.4792975072921721, 1.4785509775442356, 1.4786170225407778, 1.4793160180293166, 1.4785077457488982, 1.4761183971026812, 1.4772437043281506, 1.4762220685161762, 1.4771273441152024]
Training time: 1631.916251897812s


In [ ]:
print('best_accuracy: ',best_accuracy)

In [ ]:
best_accuracy